This notebook shows how to load a labelfield in a `.mha` file and relabel it to have consecutive label indices

In [ ]:
import SimpleITK
import numpy
import os
import csv

## Relabeling

Load image

In [ ]:
imgOld = SimpleITK.ReadImage("brain_segmentation_old.mha")

Convert image to numpy

In [ ]:
ndaOld = SimpleITK.GetArrayFromImage(imgOld)

Find unique values, i.e., label indices

In [ ]:
_indices = numpy.unique(ndaOld)
_indices

Load colorfile into dictionary

In [ ]:
fid = open("colorfile_old.txt", "r")
reader = csv.reader(fid)

dictRgbOld = {}
for line in reader:
    dictRgbOld[int(line[0])] = [line[1], line[2], line[3], line[4]]
fid.close()

Clean colorfile dictionary (in case there are pixels with labels that don't apply)

In [ ]:
for idx in _indices:
    if idx not in dictRgbOld.keys():
        ndaOld[ndaOld==idx] = 0
        
_indices = numpy.unique(ndaOld)
_indices

Relabel

In [ ]:
counterIdxNew = 0
dictRgbNew = {}

for idx in _indices:
    ndaOld[ndaOld==idx] = counterIdxNew
    dictRgbNew[counterIdxNew] = dictRgbOld[idx]
    counterIdxNew += 1

Create new `SimpleITK.Image` out of cleaned numpy array

In [ ]:
imgNew = SimpleITK.GetImageFromArray(ndaOld)
imgNew.SetSpacing(imgOld.GetSpacing())
imgNew.SetOrigin(imgOld.GetOrigin())

Save new colorfile

In [ ]:
fid = open("colorfile_new.txt", "wb")
writer = csv.writer(fid)

for idx in dictRgbNew.keys():
    l = list(dictRgbNew[idx])
    l.insert(0, idx)
    writer.writerow(str.encode(l))
fid.close()

Save new `.mha` file

In [ ]:
SimpleITK.WriteImage(imgNew, "brain_segmentation_new.mha", useCompression=True)